<a href="https://colab.research.google.com/github/GuilhermeVCCdaSilva/ProjetoBD/blob/main/ProjetoBigData_GuilhermeSilvaMarineFournieGon%C3%A7aloAlves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://www.estbarreiro.ips.pt/Content/images/logo-ESTB.png" align="right" alt="EsT Logo circle logo" style="height: 200px; width:300px;"/>

# Projeto Final Big Data 
* __Guilherme Silva__
* __Marine Emilie Fournier__
* __Gonçalo Alves__

In [29]:
# Installing required packages
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Creating a SparkSession
Uma **SparkSession** é o ponto de entrada principal para trabalhar com dados no Spark. Ele é responsável por gerenciar a configuração do Spark, criar RDDs (Resilient Distributed Datasets) e DataFrames, registrar tabelas temporárias e gerenciar o conjunto de recursos de cluster que estão sendo usados.

A SparkSession é a classe principal do Spark SQL, que é o módulo do Spark que permite trabalhar com dados estruturados e relacionais. Ele permite que você execute consultas SQL, manipule DataFrames e extraia metadados. Ele também oferece acesso às bibliotecas de processamento de fluxo de dados (como o DataFrame API e o SQL) e às bibliotecas de aprendizado de máquina (como o MLlib) do Spark.

Uma SparkSession é criada usando o SparkSession.builder. É possível configurar a sessão com várias opções, como o nome do aplicativo, o endereço do cluster, o número de núcleos para usar e a quantidade de memória. Além disso, é possível usar a SparkSession para gerenciar configurações globais, como as configurações de memória e de arquivo de log.

.config("spark.memory.offHeap.enabled","true") é uma configuração utilizada para habilitar o uso de memória fora da heap (off-heap) no Spark. Quando essa configuração é definida como "true", o Spark pode usar memória fora do heap do JVM para armazenar os dados e realizar operações. Isso pode ser útil quando o heap do JVM não é suficiente para armazenar todos os dados, permitindo que o Spark use mais memória do que o heap para melhorar o desempenho das operações.

.config("spark.memory.offHeap.size","10g") é uma configuração utilizada para definir a quantidade de memória fora da heap (off-heap) que o Spark pode usar. O valor "10g" indica que o Spark pode usar até 10 gigabytes de memória fora da heap. Esse valor pode ser ajustado de acordo com o tamanho dos dados e as necessidades do seu aplicativo.

In [ ]:
from pyspark.sql import SparkSession
import findspark

findspark.init()

spark = SparkSession.builder.appName("Pyspark Tutorial")\
                    .config("spark.memory.offHeap.enabled","true")\
                    .config("spark.memory.offHeap.size","10g")\
                    .getOrCreate()
spark                    

Carregamento dos dados do DataSet
O dataset pode ser obtido em https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work

In [ ]:
!curl https://raw.githubusercontent.com/GuilhermeVCCdaSilva/ProjetoBD/main/Absenteeism_at_work.csv >> Absenteeism_at_work.csv
df = spark.read.csv('Absenteeism_at_work.csv',header=True,sep=";")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45232  100 45232    0     0   204k      0 --:--:-- --:--:-- --:--:--  204k


In [ ]:
df.show(10)

+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+
| ID|Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time|Age|Work load Average/day |Hit target|Disciplinary failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Height|Body mass index|Absenteeism time in hours|
+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+
| 11|                26|               7|              3|      1|                   289|                        

#Este DataSet contém várias colunas tais como:

1.  ID: Identificador único para cada registo.
2.    Razão para a ausência: Razão para a ausência do funcionário no trabalho.
3.    Mês da ausência: O mês em que o funcionário estava ausente.
4.    Dia da semana: O dia da semana em que o funcionário estava ausente.
5.    Estações: A estação do ano em que o funcionário estava ausente.
6.    Despesas de transporte: As despesas de transporte incorridas pelo funcionário ao viajar para o trabalho.
7.    Distância da residência para o trabalho: A distância entre a residência do funcionário e o local de trabalho.
8.    Tempo de serviço: O período de tempo em que o funcionário está a trabalhar na empresa.
9.    Idade: A idade do funcionário.
10.    Carga de trabalho média/dia: A carga de trabalho média do funcionário por dia.
11.    Alvo atingido: Se o funcionário atingiu ou não o alvo
12.    Falha disciplinar: Se o funcionário cometeu alguma falha disciplinar ou não.
12.    Educação: O nível de educação do funcionário.
14.    Filhos: O número de filhos que o funcionário tem.
15.    Bebedor social: Se o funcionário bebe socialmente ou não.
16.    Fumador social: Se o funcionário fuma socialmente ou não.
17.    Animal de estimação: Se o funcionário tem algum animal de estimação ou não.
18.    Peso: O peso do funcionário.
19.    Altura: A altura do funcionário.
20.    Índice de massa corporal: O índice de massa corporal do funcionário.
21.    Tempo de ausência em horas: O número de horas que o funcionário ficou ausente.

#Exploratory Data Analysis

- Check the number of rows and columns in the dataset

In [ ]:
print("Number of rows:", df.count())
print("Number of columns:", len(df.columns))

Number of rows: 740
Number of columns: 21


- Check the number of missing values in each column

In [ ]:
# create a list to store the number of missing values in each column
missing_values = []

# iterate through the columns in the dataset
for col in df.columns:
    missing_values.append((col, df.filter(df[col].isNull()).count()))

# print the missing values
for col, val in missing_values:
    if val == 0:
        print("{} : No missing values".format(col))
    else:
        print("{} : {} missing values".format(col, val))

ID : No missing values
Reason for absence : No missing values
Month of absence : No missing values
Day of the week : No missing values
Seasons : No missing values
Transportation expense : No missing values
Distance from Residence to Work : No missing values
Service time : No missing values
Age : No missing values
Work load Average/day  : No missing values
Hit target : No missing values
Disciplinary failure : No missing values
Education : No missing values
Son : No missing values
Social drinker : No missing values
Social smoker : No missing values
Pet : No missing values
Weight : No missing values
Height : No missing values
Body mass index : No missing values
Absenteeism time in hours : No missing values


- check the data types of the columns

In [ ]:
df.dtypes

[('ID', 'string'),
 ('Reason for absence', 'string'),
 ('Month of absence', 'string'),
 ('Day of the week', 'string'),
 ('Seasons', 'string'),
 ('Transportation expense', 'string'),
 ('Distance from Residence to Work', 'string'),
 ('Service time', 'string'),
 ('Age', 'string'),
 ('Work load Average/day ', 'string'),
 ('Hit target', 'string'),
 ('Disciplinary failure', 'string'),
 ('Education', 'string'),
 ('Son', 'string'),
 ('Social drinker', 'string'),
 ('Social smoker', 'string'),
 ('Pet', 'string'),
 ('Weight', 'string'),
 ('Height', 'string'),
 ('Body mass index', 'string'),
 ('Absenteeism time in hours', 'string')]

- Convert data type string to double

In [ ]:
from pyspark.sql.functions import col
for column in df.columns:
    df = df.withColumn(column, col(column).cast("double"))

In [35]:
df.dtypes

[('ID', 'double'),
 ('Reason for absence', 'double'),
 ('Month of absence', 'double'),
 ('Day of the week', 'double'),
 ('Seasons', 'double'),
 ('Transportation expense', 'double'),
 ('Distance from Residence to Work', 'double'),
 ('Service time', 'double'),
 ('Age', 'double'),
 ('Work load Average/day ', 'double'),
 ('Hit target', 'double'),
 ('Disciplinary failure', 'double'),
 ('Education', 'double'),
 ('Son', 'double'),
 ('Social drinker', 'double'),
 ('Social smoker', 'double'),
 ('Pet', 'double'),
 ('Weight', 'double'),
 ('Height', 'double'),
 ('Body mass index', 'double'),
 ('Absenteeism time in hours', 'double')]

- Check the statistics of numerical columns

In [33]:
df.describe().toPandas()

,summary,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,count,1480,1480,1480,1480,1480,1480,1480,1480,1480,...,1480,1480,1480,1480,1480,1480,1480,1480,1480,1480
1,mean,18.017567567567568,19.216216216216218,6.324324324324325,3.9148648648648647,2.5445945945945945,221.32972972972973,29.63108108108108,12.554054054054054,36.45,...,0.05405405405405406,1.2918918918918918,1.018918918918919,0.5675675675675675,0.07297297297297298,0.745945945945946,79.03513513513514,172.11486486486487,26.677027027027027,6.924324324324324
2,stddev,11.017520721194476,8.430554350720568,3.4351250428068756,1.4211940082345906,1.1114551240316193,66.92958513030392,14.831771771021184,4.3833907792076365,6.476581832880566,...,0.22620076358559088,0.6730104039729131,1.0981175946609893,0.49558104296288513,0.2601800475555146,1.3178125573177155,12.878854391919235,6.032953957248968,4.284003211385137,13.326490578245464
3,min,1.0,0.0,0.0,2.0,1.0,118.0,5.0,1.0,27.0,...,0.0,1.0,0.0,0.0,0.0,0.0,56.0,163.0,19.0,0.0
4,max,36.0,28.0,12.0,6.0,4.0,388.0,52.0,29.0,58.0,...,1.0,4.0,4.0,1.0,1.0,8.0,108.0,196.0,38.0,120.0


- Check the correlation between columns

In [ ]:
corr = df.stat.corr("Month of absence", "Transportation expense")
print(corr)

0.13752456639267643


- Check the number of unique values in each column

In [31]:
for col in df.columns:
    print("Unique values in column '{}':".format(col), df.select(col).distinct().count())

Unique values in column 'ID': 37
Unique values in column 'Reason for absence': 29
Unique values in column 'Month of absence': 14
Unique values in column 'Day of the week': 6
Unique values in column 'Seasons': 5
Unique values in column 'Transportation expense': 25
Unique values in column 'Distance from Residence to Work': 26
Unique values in column 'Service time': 19
Unique values in column 'Age': 23
Unique values in column 'Work load Average/day ': 39
Unique values in column 'Hit target': 14
Unique values in column 'Disciplinary failure': 3
Unique values in column 'Education': 5
Unique values in column 'Son': 6
Unique values in column 'Social drinker': 3
Unique values in column 'Social smoker': 3
Unique values in column 'Pet': 7
Unique values in column 'Weight': 27
Unique values in column 'Height': 15
Unique values in column 'Body mass index': 18
Unique values in column 'Absenteeism time in hours': 20


- check the distribution of each column

In [32]:
for col in df.columns:
    df.select(col).groupBy(col).count().show()

+----+-----+
|  ID|count|
+----+-----+
| 8.0|    4|
| 7.0|   12|
|29.0|   10|
|35.0|    2|
|null|    1|
|18.0|   32|
| 1.0|   46|
|34.0|  110|
|25.0|   20|
|36.0|   68|
| 4.0|    2|
|23.0|   16|
|31.0|    6|
|11.0|   80|
|21.0|    6|
|14.0|   58|
|22.0|   92|
| 3.0|  226|
|19.0|    6|
|28.0|  152|
+----+-----+
only showing top 20 rows

+------------------+-----+
|Reason for absence|count|
+------------------+-----+
|               8.0|   12|
|               0.0|   86|
|               7.0|   30|
|              null|    1|
|              18.0|   42|
|               1.0|   32|
|              25.0|   62|
|               4.0|    4|
|              23.0|  298|
|              11.0|   52|
|              21.0|   12|
|              14.0|   38|
|              22.0|   76|
|              19.0|   80|
|               3.0|    2|
|              28.0|  224|
|               2.0|    2|
|              17.0|    2|
|              27.0|  138|
|              10.0|   50|
+------------------+-----+
only showing t